# 0. what for
1. m和v固定的时候，real data上是否有多组对应的b与d？以此来研究当m和v固定时，b与d对于revenue的影响
# 1.

In [4]:
import os

import numpy as np
import pandas as pd
from math import sqrt
import torch
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 人工数据集or真实数据集
ARTIFICIAL = False
# 是否包含voucher
with_vou = False

seed = 307

noise_pct = 0.05  # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct
SELECTED_thrd = 2

target_path = "../data/targets_all"

data_key_path = "../data/target_datakey_all.csv"
target_ER_path = "../data/revenue/target_ER.csv"
GT_1_ER_path = "../data/revenue/target_GT_1_ER.csv"
GT_2_ER_path = "../data/revenue/target_GT_2_ER_seed="+str(seed)+".csv"
MDN_ER_path = "../data/revenue/target_MDN_ER_seed="+str(seed)+".csv"


if with_vou:
    file_with_vou = ""
else:
    file_with_vou = "_wo_vou"

# Data path
if ARTIFICIAL:

    target_path_metric = "../data/artificial_targets_v2_" + "noise=" + str(noise_pct)

    NLL_metric_path = "../data/GT_metric/NLL_metric_GT_Tgt=1_e30_all_" + "artificial_targets_v2_" + "noise=" + str(
    noise_pct) + "_seed=" + str(seed) + ".csv"

else:

    target_path_metric = "../data/targets_all"
    NLL_metric_path = "../data/GT_metric/NLL_metric_GT_Tgt=1_e30_seed="+str(seed)+".csv"

# output path
setting_list = "../data/revenue/setting_bd_"+str(SELECTED_thrd)+".csv"
# 只保留同一个(m,v)对应了多组形成对比的(b,d)的config
setting_selected_list = "../data/revenue/setting_bd_"+str(SELECTED_thrd)+"_selected.csv"
# by hand, 筛选
setting_analysis = "../data/revenue/setting_bd_"+str(SELECTED_thrd)+"_selected_analysis_seed="+str(seed)+".csv"

col_mv = ['desc','retail']
col_ER = ['revenue','retail','odds']
col_config = ['desc','bidincrement','bidfee','retail','flg_endprice']


In [5]:
data_key = pd.read_csv(data_key_path,encoding="utf-8")
target_ER = pd.read_csv(target_ER_path,encoding="utf-8")
GT_1_ER = pd.read_csv(GT_1_ER_path,encoding="utf-8")
GT_2_ER = pd.read_csv(GT_2_ER_path,encoding="utf-8")
MDN_ER = pd.read_csv(MDN_ER_path,encoding="utf-8")
LEN_ALL = data_key.shape[0]
data_key.head()

,desc,bidincrement,bidfee,retail,flg_endprice
0,Sony Ericsson S500i Unlocked Mysterious Green,0.15,0.75,499.99,0
1,PSP Slim & Lite Sony Piano Black,0.15,0.75,169.99,0
2,iPod Touch Apple 8GB with Software Upgrade,0.15,0.75,299.99,0
3,Logitech Cordless Wave Keyboard and Mouse,0.15,0.75,89.99,0
4,SanDisk Cruzer Contour 4GB,0.15,0.75,59.99,0


In [6]:


def setup_seed(seed):
    """
    Set seed
    Args:
        seed:
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(seed)

seed

307

In [219]:
train_pct = 0.7
vali_pct = 0.2

# 使用全部的data
shuffled_indices = np.random.permutation(data_key.shape[0])

train_idx = shuffled_indices[:int(train_pct*LEN_ALL)]
# train_idx = shuffled_indices
tmp = int((train_pct+vali_pct)*LEN_ALL)
val_idx = shuffled_indices[int(train_pct*LEN_ALL):tmp]
test_idx = shuffled_indices[tmp:]
test_idx

array([ 207, 1191,   62, 1260, 1250, 1166, 1230, 1049, 1238,  660,  872,
        215,  662, 1018,  256,  722,   18, 1174, 1258,  507, 1074,  771,
        687,  762,  107,   10,  199,  260,  761,  859,  705,  430,  407,
        288,   80,  514,  368,  888,  158,  637,  875,   37,  695,  844,
        703,  421,  795,   28,  912,   53,  316,  892,    0,  150,  201,
        165,  425,  714,  457, 1186,  806,  878,  346,  579,  909, 1251,
        697, 1109,   70, 1256,  284, 1003, 1035,  254,  206,  509,  379,
         54,  267,  327,  777, 1147,  924,  478,   25,  869,    4,  931,
        749, 1000,  161,  799,  857,  311, 1214,  961,  883,  427, 1006,
        245,   91, 1187, 1066,  981, 1046,  527,  782,  486,  440,  273,
         97,  624,  925,  125,  108,   29,  960,  467,  303,  958,  341,
        455,  371, 1130,  319,  578,  932,  689])

# 2. 筛选m和v，对应的b与d
1. 有多少组m和v？他们最多出现了几次？
    - 出现次数>=SELECTED_thrd

In [220]:
data_mv = data_key.groupby(by=col_mv,as_index=False)
data_mv_df = pd.DataFrame(data_mv.size())
# 按照size排序
# data_mv_df.sort_values(by=['size'],ascending=False,inplace=True)
# data_mv_df.head()
data_mv_selected = data_mv_df[data_mv_df['size']>=SELECTED_thrd]
data_mv_selected.reset_index(inplace=True,drop=True)
print(f" ‘size’ > {SELECTED_thrd}的一共有 {data_mv_selected.shape[0]}")
# data_mv_selected

 ‘size’ > 2的一共有 88


In [221]:
data_mv_selected.head()

,desc,retail,size
0,"$1,000 Cash!",1000.00,2
1,$799 iPhone 3G Gift Card,799.99,2
2,$80 Cash!,80.00,2
3,20 Bids Voucher,12.00,4
4,300 Bids Voucher,180.00,4


2. 获得这些mv对应的bd值
3. 可以发现，基本上都是一个bidfee对应2个bid increment的情况

In [222]:
mvbd = pd.DataFrame()

for i in range(data_mv_selected.shape[0]):
    key = (data_mv_selected.iloc[i,0],data_mv_selected.iloc[i,1])
    mvbd = pd.concat([mvbd,data_mv.get_group(key)],axis=0,ignore_index=True)
mvbd.shape

(192, 5)

In [223]:
# mvbd.to_csv(setting_list,encoding="utf-8",index=False,header=True)

# 3. 读入setting，找到他们的revenue以及rmse
## 3.1 找到rmse
1. setting来源于selected file
2. 要先找到idx.

In [224]:
mvbd = pd.read_csv(setting_selected_list,encoding="utf-8")

In [225]:
data_mv_windex = data_key.reset_index(drop=False)
mvbd_selected = pd.merge(mvbd,data_mv_windex,how="left")
idx = mvbd_selected['index'].tolist()

In [226]:
mvbd_selected

,desc,bidincrement,bidfee,retail,flg_endprice,index
0,"$1,000 Cash!",0.00,0.75,1000.00,1,21
1,"$1,000 Cash!",0.01,0.75,1000.00,0,407
2,$799 iPhone 3G Gift Card,0.06,0.60,799.99,0,927
3,$799 iPhone 3G Gift Card,0.02,0.60,799.99,0,928
4,$80 Cash!,0.00,0.75,80.00,1,20
...,...,...,...,...,...,...
122,UFC 2009: Undisputed (PS3),0.12,0.60,52.99,0,835
123,Verizon VZ-V300AM-2 DECT 6.0 Cordless Phone,0.12,0.60,119.99,0,722
124,Verizon VZ-V300AM-2 DECT 6.0 Cordless Phone,0.24,0.60,119.99,0,778
125,Wii Fit | Nintendo WiiFit + Balance Board,0.15,0.75,89.99,0,183


In [227]:
def get_rmse(target,data):
    target.reset_index(inplace=True,drop=True)
    data.reset_index(inplace=True,drop=True)

    rmse = sqrt(mean_squared_error(data.odds, target.odds))
    mae = mean_absolute_error(data.odds, target.odds)
    # rmse = np.sqrt(np.sum( (target.odds-data.odds)*(target.odds-data.odds))/target.shape[0] )
    return rmse,mae

In [228]:
print(f"GT-1")
get_rmse(target_ER.iloc[idx,:], GT_1_ER.iloc[idx,:])

GT-1


(5.19983091060939, 2.217875120788871)

In [229]:
print(f"GT-2")
get_rmse(target_ER.iloc[idx,:], GT_2_ER.iloc[idx,:])

GT-2


(9.185248756399133, 3.95317488689578)

In [230]:
print(f"MDN")
get_rmse(target_ER.iloc[idx,:], MDN_ER.iloc[idx,:])

MDN


(4.791967493989202, 1.8434802776963384)

## 3.2 找到revenue

In [231]:
target_ER = pd.read_csv(target_ER_path,encoding="utf-8")
GT_1_ER = pd.read_csv(GT_1_ER_path,encoding="utf-8")
GT_2_ER = pd.read_csv(GT_2_ER_path,encoding="utf-8")
MDN_ER = pd.read_csv(MDN_ER_path,encoding="utf-8")

In [232]:
# tmp = target_ER
mvbd_selected['target'] = round(target_ER.odds,2)
mvbd_selected['GT1'] = GT_1_ER.odds
mvbd_selected['GT2'] = round(GT_2_ER.odds,2)
mvbd_selected['MDN'] = round(MDN_ER.odds,2)

## 3.3 是否位于test set

In [233]:
mvbd_selected['flg_test_set'] = [i in test_idx for i in mvbd_selected['index']]

In [234]:
mvbd_selected

,desc,bidincrement,bidfee,retail,flg_endprice,index,target,GT1,GT2,MDN,flg_test_set
0,"$1,000 Cash!",0.00,0.75,1000.00,1,21,0.63,1,1.53,1.90,False
1,"$1,000 Cash!",0.01,0.75,1000.00,0,407,2.38,1,1.54,1.36,True
2,$799 iPhone 3G Gift Card,0.06,0.60,799.99,0,927,2.03,1,1.59,5.52,False
3,$799 iPhone 3G Gift Card,0.02,0.60,799.99,0,928,2.30,1,1.54,0.97,False
4,$80 Cash!,0.00,0.75,80.00,1,20,0.82,1,1.40,2.59,False
...,...,...,...,...,...,...,...,...,...,...,...
122,UFC 2009: Undisputed (PS3),0.12,0.60,52.99,0,835,2.32,1,1.73,3.16,False
123,Verizon VZ-V300AM-2 DECT 6.0 Cordless Phone,0.12,0.60,119.99,0,722,1.97,1,1.77,3.10,True
124,Verizon VZ-V300AM-2 DECT 6.0 Cordless Phone,0.24,0.60,119.99,0,778,1.94,1,61.04,1.24,False
125,Wii Fit | Nintendo WiiFit + Balance Board,0.15,0.75,89.99,0,183,6.98,1,1.77,1.70,False


In [235]:
setting_analysis

'../data/revenue/setting_bd_2_selected_analysis_seed=307.csv'

In [236]:
mvbd_selected.to_csv(setting_analysis,encoding="utf-8",header=True,index=False)

# 4. 具体分析
## 4.1 调查每个setting的data records数量
1. 平均213

In [7]:
seed_list = [3,31,204,223,407]
data_record_lst = []
all_target_file = os.listdir(target_path)

In [8]:

for seed_i in seed_list:
    record_cnt = 0
    setting_analysis = "../data/revenue/setting_bd_"+str(SELECTED_thrd)+"_selected_analysis_seed="+str(seed_i)+file_with_vou+".csv"
    mvbd_anaysis = pd.read_csv(setting_analysis,encoding="utf-8")
    print(f"seed={seed_i}")
    print(setting_analysis)

    for j in range(mvbd_anaysis.shape[0]):

        setting_j = mvbd_anaysis.loc[j,col_config]
        # 根据setting找到key
        idx = data_key[((data_key['desc'] == setting_j['desc']) &
            (data_key['bidincrement'] == setting_j['bidincrement']) &
            (data_key['bidfee'] == setting_j['bidfee']) &
            (data_key['retail'] == setting_j['retail']) &
            (data_key['flg_endprice'] == setting_j['flg_endprice']))].index[0]
        # 获取idx对应的file以及长度
        target_data_path = os.path.join(target_path,all_target_file[idx])
        target_data = pd.read_csv(target_data_path,encoding="utf-8")
        record_cnt += target_data.shape[0]
    # records总数/
    recond_avg = int(record_cnt / mvbd_anaysis.shape[0])
    data_record_lst.append(recond_avg)

seed=3
../data/revenue/setting_bd_2_selected_analysis_seed=3_wo_vou.csv
seed=31
../data/revenue/setting_bd_2_selected_analysis_seed=31_wo_vou.csv
seed=204
../data/revenue/setting_bd_2_selected_analysis_seed=204_wo_vou.csv
seed=223
../data/revenue/setting_bd_2_selected_analysis_seed=223_wo_vou.csv
seed=407
../data/revenue/setting_bd_2_selected_analysis_seed=407_wo_vou.csv


In [9]:
print("data record数量：")
data_record_lst, np.mean(data_record_lst)

data record数量：


([197, 97, 67, 75, 214], 130.0)

## 4.2 [bid fee不变，从bid inc的角度分析]

In [189]:
mvbd_anaysis = pd.read_csv(setting_analysis,encoding="utf-8")
mvbd_anaysis.GT2 = round(mvbd_anaysis.GT2,2)
mvbd_anaysis.MDN = round(mvbd_anaysis.MDN,2)
mvbd_anaysis.target = round(mvbd_anaysis.target,2)
mvbd_anaysis.to_csv(setting_analysis,encoding="utf-8",header=True,index=False)